# C. Coelho, M. F. P. Costa, and L.L. Ferrás, "The role of adaptive activation functions in Fractional Physics-Informed Neural Networks" [[1]](https://arxiv.org)

In [ ]:
%%capture
!python setup.py install

In [ ]:
%%capture
pip install scikit-optimize matplotlib pandas

In [ ]:
%%capture
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from scipy.special import gamma
import matplotlib.pyplot as plt
import pandas as pd

import deepxde as dde
from deepxde.backend import tf

## Introduction

In this notebook we want to study the effect of adaptive activation functions on the convergence rate and solution
accuracy of fPINNs [[2]](https://arxiv.org/abs/1811.08967v1). 

We consider the numerical solution of the fractional Laplacian equation (FLE),

\begin{equation}
    (-\Delta)^{\alpha/2}u(x)=f(x),~~~u(0)=u(1)=0,~~~x\in(0,1),~\alpha\in(1,2) \tag{1}
\end{equation}
where $(-\Delta)^{\alpha/2}$ is the fractional Laplacian operator [[3]](https://arxiv.org). 

<span style="color:red">add link to go to icnaam 2021 paper</span>

We introduced one trainable parameter per layer to the activation functions and solved the
1D Fractional Laplacian Equation with a non-smooth analytical solution. 
Also, the aim of the experiments was to analyse the limitations of fPINNs when solving the FLE with a series of gradually smaller non-integer order α values. 

Taking $f(x)=\left(2\cos\left(\frac{\pi\alpha}{2}\right)\right)\Gamma (\alpha + 2)x$ comes $u(x)=x(1-x^2)^{\alpha /2}$ as the numerical solution of equation (1). This solution is non-smooth being a more extreme case so greater difficulties, in in approximating the fractional operator (through the difference scheme) and in solving the equation by fPINN, are expected as seen in [[3]](https://arxiv.org).


## Experiments description

We considered a feed-forward NN with 5 hidden layers and a configuration of 128-64-32-16-8 neurons.
In our work training waas done with different combinations of hyperparameters:
- activation function: tanh, silu;
- optimizer: L-BFGS;
- initializer (weights): Glorot Normal with mean and standard deviation given respectively by $0$ and  $\sqrt{\frac{2}{n^{k}+n^{k-1}}}$ ($n^j$ is the number of neurons in the layer $j=k-1,k$) and uniform distributions given by $U(-1,1)$ and $U(-2,2)$;
- alpha: 1.9,1.8,1.7,1.6,1.5,1.4,1.3,1.2;

In [ ]:
def NN(activation, optimizer, initializer, alpha):
    
    def fle(x, y, int_mat):
        # defining the fractional Laplacian equation(1)
        if isinstance(int_mat, (list, tuple)) and len(int_mat) == 3:
            int_mat = tf.SparseTensor(*int_mat)
            lhs = tf.sparse_tensor_dense_matmul(int_mat, y)
        else:
          
          lhs = tf.matmul(int_mat, y)
         
        lhs /= 2 * np.cos(alpha * np.pi / 2)
        rhs = gamma(alpha + 2) * x
        return lhs - rhs[: tf.size(lhs)]

    def NNSol(x):
        # defining the forced non-smooth numerical solution
        return x * (np.abs(1 - x**2)) ** (alpha / 2)
        

    geom = dde.geometry.Interval(0, 1)
    bc = dde.DirichletBC(geom, NNSol, lambda _, on_boundary: on_boundary)


    data = dde.data.FPDE(geom, fle, alpha, bc, [101], meshtype="static", solution=NNSol, num_domain=40)
    
    # network architecture
    net = dde.maps.FNN([1] + [128] + [64] + [32] + [16] + [8] + [1], activation, initializer)
    net.apply_output_transform(lambda x, y: x * (1 - x) * y)

    # training
    model = dde.Model(data, net)
    model.compile(optimizer, metrics=["l2 relative error"])
    losshistory, train_state = model.train(epochs=50000)

    # testing
    X = geom.random_points(1000)
    y_true = NNSol(X)
    y_pred = model.predict(X)
    
    model.print_model()
    
    return losshistory 

To avoid being trapped in a local optimal solution we implemented a multi-start strategy

In [ ]:
def multiStart(n, activation, optimizer, initializer, l_alpha):
    res = {}
    for j in range(len(l_alpha)):
        res[str(l_alpha[j])] = []
        for i in range(n):
            res[str(l_alpha[j])].append(NN(activation, optimizer, initializer, l_alpha[j]))
    return res

In [ ]:
# list with alpha values
l_alpha = [1.9,1.8,1.7,1.6,1.5,1.4,1.3,1.2]

## Training using different weight's initialization

### Glorot Normal

#### Tanh

In [ ]:
rGlorotTanh = multiStart(5, 'LAAF-1 tanh', 'L-BFGS',"Glorot normal", l_alpha)

#### Silu

In [ ]:
rGlorotSilu = multiStart(5, 'LAAF-1 silu', 'L-BFGS',"Glorot normal",l_alpha)

### Random uniform [-1,1]

In [ ]:
initializer = tf.keras.initializers.RandomUniform(minval=-1, maxval=1, seed=None)

#### Tanh

In [ ]:
rRandom1Tanh = multiStart(5, 'LAAF-1 tanh', 'L-BFGS',initializer,l_alpha)

#### Silu

In [ ]:
rRandom1Silu = multiStart(5, 'LAAF-1 silu', 'L-BFGS',initializer,l_alpha)

### Random uniform [-2,2]

In [ ]:
initializer = tf.keras.initializers.RandomUniform(minval=-2, maxval=2, seed=None)

#### Tanh

In [ ]:
rRandom2Tanh = multiStart(5, 'LAAF-1 tanh', 'L-BFGS',initializer,l_alpha)

#### Silu

In [ ]:
rRandom2Silu = multiStart(5, 'LAAF-1 silu', 'L-BFGS',initializer,l_alpha)

## Process and present the results

we can organize the L2 relative errors by alpha value in a dataframe by using simple preprocessing

In [ ]:
def organize(results):
    for k, v in results.items():
        results[k] = list(map(lambda x: x.metrics_test[1][0], v))
    print(pd.DataFrame.from_dict(results))

In [ ]:
organize(rGlorotTanh)

In [ ]:
organize(rGlorotSilu)

In [ ]:
organize(rRandom1Tanh)

In [ ]:
organize(rRandom1Silu)

In [ ]:
organize(rRandom2Tanh)

In [ ]:
organize(rRandom2Silu)